In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

import src.assumptions as A
from src import (
    demand_model,
    matplotlib_style,  # noqa: F401
    storage_model,
    supply_model,
)
from src.units import Units as U

# storage model plots

In [ ]:
renewable_capacity = 300
demand_df = demand_model.predicted_demand(mode="naive", average_year=False)
df = supply_model.get_net_supply(demand_df).reset_index()

storage = storage_model.StorageModel(
    renewable_capacity=renewable_capacity * U.GW,
    max_storage_capacity=A.HydrogenStorage.CavernStorage.MaxCapacity,
    electrolyser_power=A.HydrogenStorage.Electrolysis.Power,
    dac_capacity=A.DAC.Capacity,
)
net_supply_df = storage.run_simulation(df)
results = storage.analyze_simulation_results(net_supply_df)
storage.print_simulation_results(results)

# meet L>20
plt.figure(figsize=(7, 4), dpi=200)
plt.plot(net_supply_df[f"L (TWh),RC={renewable_capacity}GW"], color="g", linewidth=0.3, label="Energy in Storage")
plt.axhline(storage.max_storage_capacity, linestyle="dashed", color="r", label="Minimum Storage Capacity")
plt.axhline(20, linestyle="dashed", color="b", label="Threshold Energy in Storage")
plt.title(f"R = {renewable_capacity} GW, Smax = {storage.max_storage_capacity} TWh")
plt.ylim(0, storage.max_storage_capacity * 1.1)
plt.xlabel("Day in 40 years")
plt.ylabel("Stored Energy (TWh)")
plt.legend()

In [ ]:
# 24GW nuclear & DAC capacity limit
plt.figure(figsize=(7, 4), dpi=200)

# blue: energy used for DAC # green: energy exceeded the limit (discarded
plt.plot(net_supply_df[f"R_ccs (TWh),RC={renewable_capacity}GW"], color="g", linewidth=0.1, label="Residual Energy")
plt.plot(net_supply_df[f"R_dac (TWh),RC={renewable_capacity}GW"], color="b", linewidth=0.1, label="DAC Energy")
plt.axhline(storage.dac_max_daily_energy, linestyle="dashed", color="r", label="Maximum DAC Energy")
plt.title("DAC Energy Utilization from Residual Energy")
plt.xlabel("Day in 40 years")
plt.ylabel("Energy (TWh)")
plt.legend(fontsize=7)

# surface plots


In [ ]:
import numpy as np

A.EnergyDemand2050 = 575 * U.TWh
A.EnergyDemand2050 = A.CB7EnergyDemand2050
CONTINGENCY_MIN_STORAGE = 20 * U.TWh
renewable_capacities = range(200, 410, 10)
electrolyser_powers = range(20, 110, 10)
max_storage = range(30, 80, 1)
Z = np.zeros((len(renewable_capacities), len(electrolyser_powers))) * np.nan
xyzs = []

df = supply_model.get_net_supply(naive_demand_scaling=True).reset_index()
for renewable_capacity in renewable_capacities:
    for electrolyser_power in electrolyser_powers:
        for storage in max_storage:
            model = storage_model.StorageModel(
                renewable_capacity=renewable_capacity * U.GW,
                max_storage_capacity=storage * U.TWh,
                electrolyser_power=electrolyser_power * U.GW,
                dac_capacity=A.DAC.Capacity,
            )

            net_supply_df = model.run_simulation(df.copy())
            results = model.analyze_simulation_results(net_supply_df)
            if results["minimum_storage"] < CONTINGENCY_MIN_STORAGE:
                continue

            Z[renewable_capacities.index(renewable_capacity), electrolyser_powers.index(electrolyser_power)] = storage
            break

In [ ]:
# meshgrid for x and y
X, Y = np.meshgrid(electrolyser_powers, renewable_capacities)

# flatten arrays and filter out NaN values for plot_trisurf
X_flat = X.flatten()
Y_flat = Y.flatten()
Z_flat = Z.flatten()

# create mask to remove NaN values
mask = ~np.isnan(Z_flat)
X_clean = X_flat[mask]
Y_clean = Y_flat[mask]
Z_clean = Z_flat[mask]

# plot the surface using trisurf to handle NaN values
fig = plt.figure(figsize=(6, 5), dpi=200, tight_layout=True)
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(X_clean, Y_clean, Z_clean, color="lightblue", alpha=1, edgecolor="black", linewidth=0.1)
ax.set_xlabel("Electrolyser Power (GW)")
ax.set_ylabel("Renewable Capacity (GW)")
ax.set_zlabel("Maximum Storage (TWh)")
ax.view_init(elev=20, azim=40)
# zoom out to make the axis smaller
ax.set_title("575 TWh Demand, 20 TWh Minimum Storage Capacity\nNaive Demand Scaling", fontsize=10)
plt.tight_layout()
plt.savefig(Path("storage_capacity_surface.png"), dpi=200)
plt.show()

In [ ]:
# make a similar plot using plotly for interactivity
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale="Viridis")])
fig.update_layout(
    title="3D Surface: Minimum Storage vs Renewable Capacity & Electrolyser Power",
    scene={"xaxis_title": "Renewable Capacity (GW)", "yaxis_title": "Electrolyser Power (GW)", "zaxis_title": "Minimum Storage (TWh)"},
)
fig.show()

In [ ]:
renewable_capacity = 350
df = supply_model.get_net_supply().reset_index()

storage = storage_model.StorageModel(
    renewable_capacity=renewable_capacity * U.GW,
    max_storage_capacity=A.HydrogenStorage.CavernStorage.MaxCapacity,
    electrolyser_power=A.HydrogenStorage.Electrolysis.Power,
    dac_capacity=A.DAC.Capacity,
)
net_supply_df = storage.run_simulation(df)
results = storage.analyze_simulation_results(net_supply_df)
results

# mt co2

In [ ]:
from src.utils import convert_energy_cost

convert_energy_cost(A.DAC.EnergyCost.Medium, A.MolecularWeightCO2) * 8

In [ ]:
convert_energy_cost(400 * U.kJ / U.mol, A.MolecularWeightCO2)